In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 1. Load the dataset
url = "wdbc.data"
columns = ['ID', 'Diagnosis'] + [f'Feature_{i}' for i in range(1, 31)]
data = pd.read_csv(url, header=None, names=columns)

# 2. Preprocess the dataset
data['Diagnosis'] = data['Diagnosis'].map({'B': 0, 'M': 1})  # Encode labels
X = data.iloc[:, 2:].values  # Features
y = data['Diagnosis'].values  # Labels

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

# 3. Define the Neural Network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification
])

# 4. Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

# 5. Evaluate the model
predictions = model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print(f"Accuracy without GA: {accuracy * 100:.2f}%")


Epoch 1/50


/home/pydevcasts/Templates/MLHub/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 542us/step - accuracy: 0.6939 - loss: 0.6098
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - accuracy: 0.9358 - loss: 0.2008
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - accuracy: 0.9701 - loss: 0.1066
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.9706 - loss: 0.0909
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accuracy: 0.9829 - loss: 0.0709
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - accuracy: 0.9908 - loss: 0.0500
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - accuracy: 0.9915 - loss: 0.0416
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.9947 - loss: 0.0436
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - accuracy: 0.9905 - loss: 0.0284
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - accuracy: 0.9935 - loss: 0.0384
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.9997 - loss: 0.0207
Epoch 12/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 1. Load the datasethttps://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
url = "wdbc.data"
columns = ['ID', 'Diagnosis'] + [f'Feature_{i}' for i in range(1, 31)]
data = pd.read_csv(url, header=None, names=columns)

# Preprocess the dataset
data['Diagnosis'] = data['Diagnosis'].map({'B': 0, 'M': 1})  # Encode labels (B: benign, M: malignant)
X = data.iloc[:, 2:].values  # Features
y = data['Diagnosis'].values  # Labels

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to categorical format
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=2)

# 2. Define the CNN model
def create_cnn(weights=None):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  # Correct method for defining input shape
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(2, activation='softmax')  # Binary classification
    ])
    if weights is not None:
        model.set_weights(weights)
    return model

# 3. Define the cost function for Genetic Algorithm
def evaluate_fitness(weights):
    model = create_cnn(weights)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train_cat, epochs=10, batch_size=16, verbose=0)  # Silent training
    predictions = model.predict(X_test)
    accuracy = accuracy_score(np.argmax(y_test_cat, axis=1), np.argmax(predictions, axis=1))
    return accuracy

def genetic_algorithm():
    population_size = 10
    num_generations = 20
    mutation_rate = 0.1

    # Initialize random weights
    population = [create_cnn().get_weights() for _ in range(population_size)]
    best_weights = None
    best_fitness = -np.inf

    for generation in range(num_generations):
        # Calculate fitness scores for the population
        fitness_scores = [evaluate_fitness(weights) for weights in population]

        # Select the top individuals (ensure valid indices are selected)
        sorted_indices = np.argsort(fitness_scores)[::-1]  # Descending order of fitness
        top_population = [population[i] for i in sorted_indices[:population_size // 2]]

        # Handle crossover and mutation
        new_population = []
        for i in range(0, len(top_population) - 1, 2):  # Process pairs only
            parent1, parent2 = top_population[i], top_population[i + 1]
            # Create child by averaging weights
            child = [np.mean([w1, w2], axis=0) for w1, w2 in zip(parent1, parent2)]
            new_population.append(child)

            # Apply mutation with some probability
            if np.random.rand() < mutation_rate:
                mutation = [w + np.random.normal(size=w.shape) * 0.1 for w in child]
                new_population.append(mutation)

        # Add the last individual if population size is odd
        if len(top_population) % 2 != 0:
            new_population.append(top_population[-1])

        # Update the population
        population = new_population

        # Track the best solution in the current generation
        best_idx = np.argmax(fitness_scores)
        if fitness_scores[best_idx] > best_fitness:
            best_fitness = fitness_scores[best_idx]
            best_weights = population[best_idx if best_idx < len(population) else -1]  # Safeguard index

        print(f"Generation {generation + 1}, Best Accuracy: {best_fitness * 100:.2f}%")

    return best_weights

# Optimize weights using Genetic Algorithm
optimal_weights = genetic_algorithm()

# 5. Evaluate the final optimized model
final_model = create_cnn(optimal_weights)
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
final_model.fit(X_train, y_train_cat, epochs=10, batch_size=16, verbose=1)

predictions = final_model.predict(X_test)
final_accuracy = accuracy_score(np.argmax(y_test_cat, axis=1), np.argmax(predictions, axis=1))
print(f"Final Optimized Accuracy with GA: {final_accuracy * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Generation 1, Best Accuracy: 98.25%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Generation 2, Best Accuracy: 98.25%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Generation 3, Best Accuracy: 98.25%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Generation 4, Best Accuracy: 98.25%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Generation 5, Best Accuracy: 98.25%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Generation 6, Best Accuracy: 98.25%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Generation 7, Best Accuracy: 98.25%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0

In [ ]:
% 1. Download and Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data';
filename = 'wdbc.data'; % Local filename for the dataset

% Download the dataset if it doesn't already exist
if ~isfile(filename)
    websave(filename, url);
    disp('Dataset downloaded successfully!');
end

% Open and read the dataset
fileID = fopen(filename, 'r');
if fileID == -1
    error('File not found or could not be opened. Ensure the file exists and the path is correct.');
end
data = textscan(fileID, '%s %s %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f', 'Delimiter', ',');
fclose(fileID);

% Preprocess the dataset
y = strcmp(data{2}, 'M'); % Encode labels ('B' -> 0, 'M' -> 1)
X = [data{3:end}]; % Features

% Normalize features
X = normalize(X);

% Split the dataset into train and test sets
num_samples = size(X, 1);
rand_indices = randperm(num_samples); % Shuffle the data
split_ratio = 0.8; % 80% training, 20% testing
train_size = round(split_ratio * num_samples);

train_indices = rand_indices(1:train_size);
test_indices = rand_indices(train_size+1:end);

X_train = X(train_indices, :);
y_train = y(train_indices, :);
X_test = X(test_indices, :);
y_test = y(test_indices, :);

% Convert labels to categorical format for training
y_train_cat = ind2vec(y_train' + 1); % For MATLAB NN Toolbox
y_test_cat = ind2vec(y_test' + 1);

% 2. Define the Neural Network (MLP)
hiddenLayerSize = 64; % Define hidden layer size
net = patternnet(hiddenLayerSize);

% Set training parameters
net.trainFcn = 'trainscg'; % Scaled Conjugate Gradient
net.performFcn = 'crossentropy'; % Cross-Entropy for classification
net.divideParam.trainRatio = 0.7;
net.divideParam.valRatio = 0.15;
net.divideParam.testRatio = 0.15;

% Train the Neural Network
[net, tr] = train(net, X_train', y_train_cat);

% Evaluate the trained model
outputs = net(X_test');
predictions = vec2ind(outputs) - 1; % Convert output vector to labels
accuracy_without_ga = sum(predictions == y_test') / numel(y_test);
fprintf('Accuracy without GA: %.2f%%\n', accuracy_without_ga * 100);

% 3. Genetic Algorithm Optimization
function accuracy = evaluateFitness(weights, net, X_train, y_train_cat, X_test, y_test)
    % Apply weights to the network
    net = setwb(net, weights);
    outputs = net(X_test');
    predictions = vec2ind(outputs) - 1; % Convert vector to labels
    accuracy = sum(predictions == y_test') / numel(y_test);
end

function bestWeights = geneticAlgorithm(net, X_train, y_train_cat, X_test, y_test)
    nvars = numel(getwb(net)); % Number of weights and biases
    fitnessFcn = @(weights) -evaluateFitness(weights, net, X_train, y_train_cat, X_test, y_test);

    % Genetic Algorithm settings
    options = optimoptions('ga', 'PopulationSize', 10, 'MaxGenerations', 20, ...
                           'MutationRate', 0.1, 'Display', 'iter');
    
    % Run Genetic Algorithm
    [bestWeights, fval] = ga(fitnessFcn, nvars, [], [], [], [], [], [], [], options);
    fprintf('Best Fitness (Accuracy): %.2f%%\n', -fval * 100);
end

% Apply Genetic Algorithm optimization
optimizedWeights = geneticAlgorithm(net, X_train, y_train_cat, X_test, y_test);

% Apply optimized weights to the network
net = setwb(net, optimizedWeights);

% Evaluate the optimized model
outputs = net(X_test');
predictions = vec2ind(outputs) - 1; % Convert vector to labels
accuracy_with_ga = sum(predictions == y_test') / numel(y_test);
fprintf('Accuracy with GA: %.2f%%\n', accuracy_with_ga * 100);


In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. ایجاد معماری شبکه عصبی کانولوشن (CNN)
def create_cnn():
    model = Sequential()

    # Input Layer: تصاویر با ابعاد 210x210x3
    model.add(tf.keras.layers.Input(shape=(210, 210, 3)))

    # Convolutional Layers: تعداد فیلترها و اندازه آنها طبق مقاله
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(24, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(24, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer: تبدیل نقشه‌های ویژگی به بردار
    model.add(Flatten())

    # Fully Connected Layer
    model.add(Dense(128, activation='relu'))  # Hidden layer
    model.add(Dense(2, activation='softmax'))  # Output layer (Binary classification: benign or malignant)

    return model

# 2. بارگذاری و پیش‌پردازش داده‌های BreakHis Dataset
# این بخش بسته به دیتاست واقعی قابل تغییر است
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    './dataset/train',
    target_size=(210, 210),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    './dataset/val',
    target_size=(210, 210),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 3. کامپایل و آموزش مدل
model = create_cnn()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=10, validation_data=validation_generator)

# 4. ارزیابی مدل
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    './dataset/test',
    target_size=(210, 210),
    batch_size=32,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


ValueError: Must provide at least one structure

In [31]:
train_generator = train_datagen.flow_from_directory(
    './dataset/train',
    target_size=(210, 210),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    './dataset/val',
    target_size=(210, 210),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Training samples: 0
Validation samples: 0


In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import tensorflow as tf

# اطمینان از استفاده از GPU
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("Using GPU for training!")
else:
    print("GPU not found! Training on CPU.")

# 1. تعریف مدل CNN
def create_cnn(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')  # طبقه‌بندی برای 10 کلاس
    ])
    return model

# 2. ارزیابی عملکرد (Fitness Function)
def evaluate_fitness(weights, model, data_generator, X_test, y_test, batch_size=32):
    model.set_weights(weights)  # تنظیم وزن‌ها
    # آموزش مدل با داده‌های Augmented
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(data_generator, epochs=1, steps_per_epoch=len(data_generator), verbose=0)
    # پیش‌بینی و محاسبه دقت
    predictions = model.predict(X_test)
    accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
    return accuracy

# 3. الگوریتم ژنتیک
def genetic_algorithm(model, data_generator, X_test, y_test, generations=10, population_size=5, mutation_rate=0.1):
    population = [model.get_weights() for _ in range(population_size)]
    best_weights = None
    best_fitness = -np.inf

    for generation in range(generations):
        # محاسبه Fitness برای جمعیت فعلی
        fitness_scores = [evaluate_fitness(weights, model, data_generator, X_test, y_test) for weights in population]
        sorted_indices = np.argsort(fitness_scores)[::-1]
        top_individuals = [population[i] for i in sorted_indices[:population_size // 2]]

        # عملیات Crossover
        new_population = []
        for i in range(len(top_individuals) - 1):
            parent1, parent2 = top_individuals[i], top_individuals[i + 1]
            child = [(w1 + w2) / 2 for w1, w2 in zip(parent1, parent2)]
            new_population.append(child)

        # عملیات Mutation
        for child in new_population:
            if np.random.rand() < mutation_rate:
                mutation = [w + np.random.normal(size=w.shape) * 0.1 for w in child]
                new_population.append(mutation)

        # اضافه کردن والدین به جمعیت جدید
        new_population.extend(top_individuals)

        # بروزرسانی جمعیت
        population = new_population[:population_size]
        best_fitness = max(fitness_scores)
        best_weights = population[np.argmax(fitness_scores)]
        print(f"Generation {generation+1}: Best Fitness (Accuracy) = {best_fitness * 100:.2f}%")

    return best_weights

# 4. آماده‌سازی داده‌ها
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0  # نرمال‌سازی داده‌ها
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)  # تبدیل برچسب‌ها به دسته‌بندی

# ایجاد Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=32)

# 5. اجرای الگوریتم
input_shape = X_train.shape[1:]
model = create_cnn(input_shape)
best_weights = genetic_algorithm(model, train_generator, X_test, y_test)

# تنظیم بهترین وزن‌ها برای مدل
model.set_weights(best_weights)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Final Test Accuracy with Optimized Weights: {accuracy * 100:.2f}%")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 111s 1us/step


/home/pydevcasts/Templates/MLHub/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1744752906.458075    6405 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/pydevcasts/Templates/MLHub/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `us

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


KeyboardInterrupt: 